# Visualisering af DMI Vejrdata i en GUI

## Formål

Du skal hente aktuelle eller historiske vejrdata fra DMI’s API, indlæse dem i en pandas DataFrame, og præsentere resultaterne i en PySide6-applikation. Applikationen skal bruge Matplotlib som backend til at vise grafer eller bruge plotly til en interaktivt dashboard.

## Trin 1: Hent data fra DMI API

- DMI stiller åbne data til rådighed via DMI’s [Open Data](https://opendatadocs.dmi.govcloud.dk/en/Getting_Started) API.
- For at få adgang, skal man oprette en konto og generere en gratis API-nøgle på [DMI API Portal](https://opendatadocs.dmi.govcloud.dk/en/Authentication)
- Lav et Python-script, der henter data (JSON-format) og gemmer det i en pandas DataFrame. [Her](https://opendatadocs.dmi.govcloud.dk/en/APIs/Meteorological_Observation_API) finder du eksempler for **Meteorological Observation API**
    - Tip: brug [**requests**](https://pypi.org/project/requests/) til at hente data.
    - Husk at konvertere tidspunkter til **datetime**.

In [12]:
import requests
import pandas as pd
url = "https://dmigw.govcloud.dk/v2/metObs/collections/observation/items"
params = {
    "api-key": "your_dmi_key",   # <- indsæt din egen nøgle
    "stationId": "06081",       # station fx København
    "parameterId": "temp_dry",  # parameter fx temperatur
    "limit": 100
}

response = requests.get(url, params=params).json()['features']
# print(pd.json_normalize(response)['geometry.coordinates'])

KeyError: 'features'

## Trin 2: Vælg et datasæt (fx temperatur over de seneste dage).

Udregn fx:

- gennemsnitstemperatur pr. dag
- højeste og laveste temperatur

## Trin 3: Lav en PySide6 GUI med Matplotlib

Byg en simpel GUI med PySide6. GUI’en skal have:
 
- En knap til at hente/loade data fra DMI.
- En figur (Matplotlib-plot) til at vise data.


## Ekstra udfordringer:
- Brug en DateTimePicker i PySide6, så man kan vælge tidsperiode.
- En dropdown-menu, hvor man kan vælge parameter (fx temperatur, nedbør).
- Tilføj mulighed for at vælge forskellige stationer (fx København, Aalborg).
- Lav et interaktivt `plotly.express` line plot af temperatur over tid.